In [1]:
import pandas as pd 
import os 

filepath = '/Users/olivialawinski/IKT/analytics/Semesterabgabe/raw_data/combined_dataset_with_ab.csv'
messdaten = pd.read_csv(filepath, sep=';', on_bad_lines='skip')


In [2]:
messdaten.head()

,EPOCH_TIME,DEVICE_NAME,SENSOR_TYPE,MEASUREMENT_VALUE,GPS_LAT,GPS_LONG,SOURCE_FOLDER
0,#[EPOCH_TIME],[DEVICE_NAME],[SENSOR_TYPE],[MEASUREMENT_VALUE],NaN,NaN,Uneben
1,1629745991519,LOCAL_PHONE,Y,61,NaN,NaN,Uneben
2,1629745991519,LOCAL_PHONE,80,SL_0,NaN,NaN,Uneben
3,1629745991519,LOCAL_PHONE,83,RT_O,NaN,NaN,Uneben
4,1629745991519,LOCAL_PHONE,81,RS_N,NaN,NaN,Uneben


In [4]:
messdaten['SENSOR_TYPE'] = pd.to_numeric(messdaten['SENSOR_TYPE'],errors = 'coerce')

# Filtern nach 20 
filtered_data = messdaten[messdaten['SENSOR_TYPE']== 20]

# aus der Spalte MEASUREMENT_VALUE den z-Wert extrahieren 
def extract_z_value(item):
    if isinstance(item,str) and "," in item:
        item = item.split(",")
        if len(item) == 3:
            return item[2]
    else:
        return item

In [5]:
# neue Spalte erstellen und die z-Werte dort speichern 
filtered_data.loc[:, "Z_VALUE"] = filtered_data["MEASUREMENT_VALUE"].map(extract_z_value)
filtered_data.sample(10)

/var/folders/r_/sf64g3216mq8k4tfr1t69nrm0000gn/T/ipykernel_15533/4269548950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, "Z_VALUE"] = filtered_data["MEASUREMENT_VALUE"].map(extract_z_value)


,EPOCH_TIME,DEVICE_NAME,SENSOR_TYPE,MEASUREMENT_VALUE,GPS_LAT,GPS_LONG,SOURCE_FOLDER,Z_VALUE
1065891,1630523881085,LOCAL_PHONE,20.0,"0.35,-0.06,0.3",NaN,NaN,Glatt/neue,0.3
719503,1630523932846,LOCAL_PHONE,20.0,"-1.12,-0.11,-1.57",NaN,NaN,Glatt,-1.57
1000641,1629227762113,LOCAL_PHONE,20.0,"0.26,0.09,-0.31",NaN,NaN,Glatt/alte,-0.31
1099218,1630523502335,LOCAL_PHONE,20.0,"0.16,-0.02,-0.25",NaN,NaN,Glatt/neue,-0.25
844860,1629225374033,LOCAL_PHONE,20.0,"0.08,0.08,0.02",NaN,NaN,Glatt,0.02
159288,1629745964997,LOCAL_PHONE,20.0,"-0.01,-0.23,0.59",NaN,NaN,Uneben/alte,0.59
193562,1629745202259,LOCAL_PHONE,20.0,"-0.85,0.06,0.23",NaN,NaN,Uneben/alte,0.23
420911,1630853765770,LOCAL_PHONE,20.0,"0.44,-0.58,0.2",NaN,NaN,Pflaster,0.2
1147534,1630526496036,LOCAL_PHONE,20.0,"0.04,-0.11,0.05",NaN,NaN,Glatt/neue,0.05
557530,1630854080309,LOCAL_PHONE,20.0,"-0.79,-0.65,-1.89",NaN,NaN,Pflaster/neue,-1.89


In [6]:
filtered_data = filtered_data.drop(columns = ['DEVICE_NAME','GPS_LAT','GPS_LONG'])
filtered_data

,EPOCH_TIME,SENSOR_TYPE,MEASUREMENT_VALUE,SOURCE_FOLDER,Z_VALUE
9,1629745991559,20.0,"-0.99,0.09,1.04",Uneben,1.04
10,1629745991559,20.0,"0.3,0.2,1.12",Uneben,1.12
11,1629745991560,20.0,"-0.92,0.61,-0.45",Uneben,-0.45
12,1629745991560,20.0,"-0.97,0.0,-0.58",Uneben,-0.58
13,1629745991560,20.0,"0.21,0.01,0.07",Uneben,0.07
...,...,...,...,...,...
1172221,1630526624656,20.0,"-0.39,-0.1,-0.26",Glatt/neue,-0.26
1172222,1630526624666,20.0,"-0.16,-0.03,0.13",Glatt/neue,0.13
1172223,1630526624676,20.0,"0.04,-0.23,0.14",Glatt/neue,0.14
1172224,1630526624686,20.0,"-0.28,0.05,0.07",Glatt/neue,0.07


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Auswahl der Merkmale
features = ['Z_VALUE']
X = filtered_data[features]
y = filtered_data['SOURCE_FOLDER']

# Aufteilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalisieren der Daten
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trainieren des k-NN Modells
k = 5  # Beispielwert für k
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# Vorhersagen treffen
y_pred = knn.predict(X_test)

# Evaluierung des Modells
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.18037810279394542
Classification Report:
                 precision    recall  f1-score   support

          Glatt       0.23      0.57      0.33     69504
     Glatt/alte       0.11      0.13      0.12     37631
     Glatt/neue       0.11      0.07      0.09     31512
       Pflaster       0.17      0.19      0.18     42553
  Pflaster/alte       0.07      0.02      0.04     21619
  Pflaster/neue       0.17      0.08      0.11     20628
   Tester/glatt       0.03      0.00      0.01     10790
Tester/pflaster       0.07      0.02      0.03      8071
  Tester/uneben       0.04      0.01      0.02     11889
         Uneben       0.15      0.07      0.10     43152
    Uneben/alte       0.08      0.02      0.04     21809
    Uneben/neue       0.06      0.02      0.03     21543

       accuracy                           0.18    340701
      macro avg       0.11      0.10      0.09    340701
   weighted avg       0.14      0.18      0.14    340701



In [11]:
from imblearn.over_sampling import SMOTE

# Balancieren der Daten durch Oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Überprüfen der neuen Klassenverteilung
print(f"Originale Klassenverteilung:\n{y_train.value_counts()}")
print(f"Neue Klassenverteilung nach SMOTE:\n{pd.Series(y_train_res).value_counts()}")

Originale Klassenverteilung:
SOURCE_FOLDER
Glatt              161736
Uneben             101123
Pflaster            98673
Glatt/alte          88415
Glatt/neue          73682
Uneben/neue         50602
Pflaster/alte       50483
Uneben/alte         50321
Pflaster/neue       48496
Tester/uneben       27188
Tester/glatt        25272
Tester/pflaster     18977
Name: count, dtype: int64
Neue Klassenverteilung nach SMOTE:
SOURCE_FOLDER
Tester/uneben      161736
Pflaster/alte      161736
Glatt              161736
Glatt/alte         161736
Uneben/neue        161736
Uneben             161736
Pflaster           161736
Glatt/neue         161736
Pflaster/neue      161736
Uneben/alte        161736
Tester/pflaster    161736
Tester/glatt       161736
Name: count, dtype: int64


In [12]:
from sklearn.model_selection import GridSearchCV

# Definition des k-NN Modells
knn = KNeighborsClassifier()

# Parameterbereich für Grid Search
param_grid = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}

# Grid Search mit Cross-Validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_res, y_train_res)

# Beste Parameter
best_params = grid_search.best_params_
print(f"Beste Parameter: {best_params}")

# Modell mit den besten Parametern trainieren
knn_best = KNeighborsClassifier(**best_params)
knn_best.fit(X_train_res, y_train_res)

# Vorhersagen und Evaluierung
y_pred_best = knn_best.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
report_best = classification_report(y_test, y_pred_best)

print(f'Improved Accuracy: {accuracy_best}')
print(f'Improved Classification Report:\n{report_best}')


Beste Parameter: {'n_neighbors': 11, 'weights': 'distance'}
Improved Accuracy: 0.19127915679730909
Improved Classification Report:
                 precision    recall  f1-score   support

          Glatt       0.24      0.60      0.34     69504
     Glatt/alte       0.12      0.13      0.12     37631
     Glatt/neue       0.11      0.06      0.08     31512
       Pflaster       0.19      0.21      0.20     42553
  Pflaster/alte       0.08      0.02      0.04     21619
  Pflaster/neue       0.17      0.10      0.12     20628
   Tester/glatt       0.05      0.01      0.02     10790
Tester/pflaster       0.07      0.01      0.02      8071
  Tester/uneben       0.05      0.00      0.01     11889
         Uneben       0.14      0.11      0.12     43152
    Uneben/alte       0.08      0.02      0.03     21809
    Uneben/neue       0.09      0.02      0.03     21543

       accuracy                           0.19    340701
      macro avg       0.12      0.11      0.09    340701
   weighted 